## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


User-User - это рекомендация товаров на основании схожести пользователей (напр. я купила товары, которые также купил другой клиент, т.е. предположительно у нас схожи вкусы - можно мне рекомендовать другие товары из его покупок, с большой долей вероятности они мне понравятся тоже)

Item-Item - рекомендации на осонове похожести товаров (рекомендации типа "С этим товаром часто покупают", т.е формируется множество объектов похожих на товар (с таким же рейтингом или, например, фильмы с такими же актерами) или часто участвующих с ним в связке (Совстречаемости) и рекомендуюся ближайшие "соседи")

Схожесть в базе для анализа таблица типа user-item из которой для анализа беруться разные данные в зависимости от метода рекомендации. Каждый из методов рекомендаций имеет свои несдостатки: для user-based рекомендаций - это растущее количество пользователей, а также рекомендации для холодных пользователей, которые еще не оставили о себе данныых для сравнения, для item-based это трудоемкость и субъективность оценок пользователей (кто-то всем ставит 2, а кто-то завышает из-за чего сложно говорить о реально схожести товаров по какаим-то характеристикам между собой). 

Функцию item_item_rec(interaction_matrix) можно использовать для user_user_rec, если транспорнировать матрицу так, чтобы по строкам получись юзеры (User), а по столбцам - товары (Item).

В Item-Item мы используем KNN (метод ближайших соседей), исползуем полностью всю матрицу, учитываем только кейсы реального взаимодействия. В ALS мы используем разложение матрицы на матрицу факторов юзеров и матрицу факторов товаров и исследуем взаимодействие матрицы ембедингов юзеров и товаров, даже если пользователь не имел взаимодействия с товаром, то эта характеристика тоже влияет на результат.

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


1. товары из той же группы, что были купленны ранее (канцелярия, аксессуары, принадлежности для авто)
2. рекомендовать какие-то новые товары (новинки по дате заведения в матрицу товаров), похожие с тем, что покупал ранее или из той же группы
3. если есть данные о пользователе, то учитывать их (дата рождения, количество детей)

Но это кажется, что-то не то... как вес можно учитывать частоту покупок товара пользователем и дату последней покупки (например, если купил один раз два года назад, то врядли купит еще, а если покупает с какой-то регулярностью, то скорее всего купит скоро). Можно ввести критерий оборачиваемости товара - как часто надо покупать следующий аналогичный, на какой период пользования расчитан товар, и предлагать снова, когда предполагаемый период пользования заканчивается. 

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


В анализе участвует только пользователь и товар, нет возможности сделать многофакторный анализ и подключить внешние фичи в качестве параметров, в качестве рузультата получаем не вероятность оценку.
Для разложения User-Item матрицы на матрицы эмбедингов юзеров и товаров необходимо, чтобы каждый пользователь сделал хотя бы одну покупку или чтобы каждый товар купили хотя бы один раз.

## Практическая часть


In [1]:
!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys


In [3]:
data = pd.read_csv('C:/Users/sklod/OneDrive/Рабочий стол/STUDY/!Теория/Q4.Рекомендательные системы/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\sklod\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)
    
def precision_at_k(recommended_list, 
                    bought_list, 
                    k=5):
    return precision(recommended_list[:k], bought_list) # K - кол-во билжайших соседей

In [7]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],
                                    recalculate_user=True)]
    return res

In [8]:
def ALS_tester(user_item_matrix, name_of_column):
    model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.001,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    use_gpu=False) 

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

    result[name_of_column] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [9]:
def test_matrix_weight (name_of_column, name_of_func, fill_value=0):
    global user_item_matrix
    global sparse_user_item
    global id_to_itemid
    global id_to_userid
    global itemid_to_id
    global userid_to_id
    
    user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc=name_of_func, 
                                  fill_value=fill_value
                                 )
    

    user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()# переведем в формат saprse matrix
    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values
    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))
    id_to_itemid = dict(zip(matrix_itemids, itemids))
    id_to_userid = dict(zip(matrix_userids, userids))
    itemid_to_id = dict(zip(itemids, matrix_itemids))
    userid_to_id = dict(zip(userids, matrix_userids))    
    
    ALS_tester(user_item_matrix, name_of_column)

In [10]:
test_matrix_weight('count', 'count' )

In [11]:
test_matrix_weight('max','max')

In [12]:
test_matrix_weight('mean','mean')

In [13]:
test_matrix_weight('nunique','nunique')

In [14]:
df = pd.DataFrame(columns=['aggfunc', 'presision_at_k'])
i = 0
for column in result.columns[2:]:
    presision = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 5)
    df.loc[i] = [column, presision]
    i += 1

df.sort_values(by='presision_at_k', ascending=False)

,aggfunc,presision_at_k
3,nunique,0.20392
2,mean,0.19109
1,max,0.18580
0,count,0.18080


Cамый высокий результат даёт "nunique", потому что nunique косвенно пересекается с популярностью товара.

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [15]:
for factor in [50, 100, 300]:
    model = ALS(factor, ...)

NameError: name 'ALS' is not defined

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

In [ ]:
!pip install hyperopt

# Use hyperopt!